<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)<br>
Aluno [Rafael Barbosa](https://www.linkedin.com/in/barbosa89/)


---

# **Tópicos**

<ol type="1">
  <li>AWS Lambda;</li>
  <li>AWS Step Functions;</li>
  <li>AWS EventBridge.</li>
</ol>

---

# **Exercícios**

## 1\. AWS Lambda

- Criar um *bucket* no AWS [S3](https://aws.amazon.com/pt/s3/) para salvar o dado original (`bronze`);

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/s3-bronze.png?raw=true)

- Criar uma função AWS [Lambda](https://aws.amazon.com/pt/lambda/) para extrair o dado original;

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/lambda-bronze-1.png?raw=true)
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/lambda-bronze-2.png?raw=true)

O código utilizado encontra-se abaixo.

In [ ]:
import json
import logging
from datetime import datetime

import boto3
import urllib3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  URL = 'https://www2.cetip.com.br/ConsultarTaxaDi/ConsultarTaxaDICetip.aspx'
  BRONZE_BUCKET = 'ebac-m38-bronze'

  client = boto3.client('s3')

  date = datetime.now().strftime('%Y-%m-%d')
  filename_json = f'stock-exchange-{date}.json'

  # -- extract

  try:
    http = urllib3.PoolManager()
    response = http.request(url=URL, method='get')
  except Exception as exc:
    raise exc
  else:
    data = json.loads(response.data.decode())
    logging.info(msg=data)

  # -- transform

  ...

  # -- load

  try:
      with open(f'/tmp/{filename_json}', mode='w', encoding='utf8') as fp:
          json.dump(data, fp)
      client.upload_file(Filename=f'/tmp/{filename_json}', Bucket=BRONZE_BUCKET, Key=filename_json)
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))

- Criar um *bucket* no AWS [S3](https://aws.amazon.com/pt/s3/) para salvar o dado transformado  (`silver`);

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/s3-silver.png?raw=true)

- Criar uma função AWS [Lambda](https://aws.amazon.com/pt/lambda/) para transformar o dado original;

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/lambda-silver-1.png?raw=true)
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/lambda-silver-2.png?raw=true)

O código utilizado encontra-se abaixo.

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  BRONZE_BUCKET = 'ebac-m38-bronze'
  SILVER_BUCKET = 'ebac-m38-silver'

  client = boto3.client('s3')

  date = datetime.now().strftime('%Y-%m-%d')
  filename_csv = f'stock-exchange-{date}.csv'
  filename_json = f'stock-exchange-{date}.json'

  # -- extract

  client.download_file(BRONZE_BUCKET, filename_json, f'/tmp/{filename_json}')

  with open(f"/tmp/{filename_json}", mode='r', encoding='utf8') as fp:
      data = json.load(fp)

  # -- transform

  data['taxa'] = data['taxa'].replace(',', '.')
  data['indice'] = data['indice'].replace('.', '').replace(',', '.')

  data['dataTaxa'] = datetime.strptime(data['dataTaxa'], '%d/%m/%Y').strftime('%Y-%m-%d')
  data['dataIndice'] = datetime.strptime(data['dataIndice'], '%d/%m/%Y').strftime('%Y-%m-%d')

  # -- load

  try:
      with open(f'/tmp/{filename_csv}', mode='w', encoding='utf8') as fp:
          fp.write(','.join([v for v in data.values()]))
      client.upload_file(Filename=f'/tmp/{filename_csv}', Bucket=SILVER_BUCKET, Key=f'data_referencia={date}/{filename_csv}')
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))

- Criar uma função AWS [Lambda](https://aws.amazon.com/pt/lambda/) para criar uma tabela no AWS [Athena](https://aws.amazon.com/pt/athena) apontando para o *bucket* do dado transformado.

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/lambda-silver-table.png?raw=true)
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/athena-create-cdi.png?raw=true)
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/athena-select-all-cdi.png?raw=true)

O código utilizado encontra-se abaixo.


In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  SILVER_BUCKET = 'ebac-m38-silver'

  query = f"""
  CREATE EXTERNAL TABLE IF NOT EXISTS cdi (
    taxa double,
    data_taxa string,
    indice double,
    data_indice string
  )
  PARTITIONED BY (
    data_referencia string
  )
  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
  WITH SERDEPROPERTIES ('separatorChar'=',')
  LOCATION 's3://{SILVER_BUCKET}/'
  """

  client = boto3.client('athena')

  # -- create

  try:
    client.start_query_execution(
      QueryString=query,
      ResultConfiguration={'OutputLocation': 's3://ebac-modulo-37-results/'}
    )
  except ClientError as exc:
    raise exc

  # -- update

  try:
    client.start_query_execution(
      QueryString='MSCK REPAIR TABLE cdi',
      ResultConfiguration={'OutputLocation': 's3://ebac-modulo-37-results/'}
    )
  except ClientError as exc:
    raise exc

  return json.dumps(dict(status=True))

## 2\. AWS Step Functions

 - Criar uma `state machine` com três *tasks*, uma com cada função lambda criada na item 1.

 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/step-config.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/step-state-machine.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/step-state-exec-1.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/step-state-exec-2.png?raw=true)

## 3\. AWS EventBridge

 - Criar um evento que inicie a *state machine* do item 2.

 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/eventbridge-create-0.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/eventbridge-create-1.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/eventbridge-create-2.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/eventbridge-create-3.png?raw=true)
 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M38%20imagens/eventbridge-create-4.png?raw=true)